In [1]:
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

In [2]:
import pandas as pd
import time
import networkx as nx
import matplotlib.pyplot as plt
from greee import et_graph

def SolveAndTransform() -> int:

## Create Reformulation Graph (node: spec, edge:transform)
## Actions: generate spec, generate instance, transform spec, solve instance
    # event: type, time, space,details
    # events: generate instance (abstract spec to constrete spec), translate (Format to format), transform ( spec to spec), solve (concrete spec to solution)
    # 
    etransform_graph = et_graph.EssenceTransformGraph()
    # test spec update to generator call
    start = time.time_ns() 

    spec = r'''find i : int(0..100)
such that
    i = 1 * 2 + 3 * 4
find a : bool
find b : bool
find c : bool
such that
    a = !(b /\ c)'''

    spec_ID = etransform_graph.add_e_node(spec)
    solution = etransform_graph.solve(spec_ID)
    solveTime =time.time_ns() - start
    parentSolutionID = hash(solution)

    # TODO upgrade logs: solution. time, size, number of nodes traversed
    
    start2 = time.time_ns()

    # Transform with GP2 (could be done in parallel)
    progName = "DeMorganTwo.gp2"

    spec2_ID= etransform_graph.transform_with_GP2(spec_ID,progName)
    transformTime = time.time_ns() - start2
    
    # Solve new spec
    solution2 = etransform_graph.solve(spec2_ID)
    instanceSolveTime = time.time_ns() - transformTime
    solutionID = hash(solution2)


    # PLOT
    pos = nx.spring_layout(etransform_graph.graph)
    nx.draw(etransform_graph.graph, pos)
    node_labels = nx.get_node_attributes(etransform_graph.graph,'file_name')
    nx.draw_networkx_labels(etransform_graph.graph, pos, node_labels)
    edge_labels = dict([((n1, n2), d['transformation']) for n1, n2, d in etransform_graph.graph.edges(data=True)])
    nx.draw_networkx_edge_labels(etransform_graph.graph, pos, edge_labels=edge_labels)
    plt.show(block=True)


    return {'parentID': spec_ID, 
                'childID': spec2_ID, 
                'trasform': progName, 
                'transformTime':transformTime, 
                'parentSolveTime' : solveTime,
                'instanceSolveTime':  instanceSolveTime,
                'parentSolutionID':parentSolutionID, 
                'solutionID':solutionID }
    

    

    


ImportError: cannot import name 'et_graph' from 'greee' (/Users/cls29/ree/experiments/../greee/__init__.py)

In [ ]:

# define columns
columns = ['parentID', 'childID', 'trasform', 'transformTime', 'parentSolveTime', 'instanceSolveTime','parentSolutionID', 'solutionID']  
data_rows = []
for i in range(1):
    data_rows.append(SolveAndTransform())
print(data_rows)
dataLogger = pd.DataFrame(data_rows,columns = columns)
#dataLogger.to_csv("experiments/crankSanityCheck.csv")

In [4]:
import os
import sys
sys.path.append('.')

from greee import EFormatGraph
spec = r'''
given a : bool
given b : int(1..10)
letting c be 5
find d : bool
    such that
        a = (!d \/ !(b>c))'''

ETG = EFormatGraph.EFGraph()
gp2 = ETG.FormToForm(spec,"Emini","GP2String")

hostFileName = "gp2/demorgTest.host"
#with open(hostFileName, 'w') as file:
    #file.write(gp2)
print(gp2)

[
(0,"unnamed~root")
(1,"given~GivenStatement")
(2,"a~Parameter")
(3,"bool~BoolDomain")
(4,"letting~NameLettingStatement")
(5,"c~Node")
(6,"5~Integer")
(7,"find~FindStatement")
(8,"d~DecisionVariable")
(9,"bool~BoolDomain")
(10,"such that~SuchThatStatement")
(11,"=~BinaryExpression")
(12,"a~ReferenceToParameter")
(13,"OR~BinaryExpression")
(14,"NOT~UnaryExpression")
(15,"d~ReferenceToDecisionVariable")
(16,"NOT~UnaryExpression")
(17,"GREATER~BinaryExpression")
(18,"b~Literal")
(19,"c~ReferenceToNamedConstant")
| 
(0,0,1,1)
(1,1,2,1)
(2,2,3,1)
(3,0,4,2)
(4,4,5,1)
(5,5,6,1)
(6,0,7,3)
(7,7,8,1)
(8,8,9,1)
(9,0,10,4)
(10,10,11,1)
(11,11,12,1)
(12,11,13,2)
(13,13,14,1)
(14,14,15,1)
(15,13,16,2)
(16,16,17,1)
(17,17,18,1)
(18,17,19,2)
]
